In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load datasets
travel_data = pd.read_csv('combined_travel_data.csv')
economic_data = pd.read_csv('combined_economic_data.csv')

# Quick look at the data
print(travel_data.head())
print(economic_data.head())


        Month  Trip ID Destination  Start date    End date  Duration (days)  \
0  2023-05-01        1  London, UK  2023-05-01  2023-05-08              7.0   
1  2023-05-01        1  London, UK  2023-05-01  2023-05-08              7.0   
2  2023-05-01        1  London, UK  2023-05-01  2023-05-08              7.0   
3  2023-05-01        1  London, UK  2023-05-01  2023-05-08              7.0   
4  2023-05-01        1  London, UK  2023-05-01  2023-05-08              7.0   

  Traveler name  Traveler age Traveler gender Traveler nationality  ...  \
0    John Smith          35.0            Male             American  ...   
1    John Smith          35.0            Male             American  ...   
2    John Smith          35.0            Male             American  ...   
3    John Smith          35.0            Male             American  ...   
4    John Smith          35.0            Male             American  ...   

  ProductPitched  PreferredPropertyStar MaritalStatus  NumberOfTrips Passp

In [3]:
# Checking for missing values
print(travel_data.isnull().sum())
print(economic_data.isnull().sum())

# Checking data types
print(travel_data.dtypes)
print(economic_data.dtypes)



Month                         0
Trip ID                       0
Destination                   0
Start date                    0
End date                      0
Duration (days)               0
Traveler name                 0
Traveler age                  0
Traveler gender               0
Traveler nationality          0
Accommodation type            0
Accommodation cost            0
Transportation type           0
Transportation cost           0
Simplified Destination        0
Max TemperatureC              1
Mean TemperatureC             1
Min TemperatureC              1
Dew PointC                    1
MeanDew PointC                1
Min DewpointC                 1
Max Humidity                  1
Mean Humidity                 1
Min Humidity                  1
Max Sea Level PressurehPa     1
Mean Sea Level PressurehPa    1
Min Sea Level PressurehPa     1
Max VisibilityKm              1
Mean VisibilityKm             1
Min VisibilityKm              1
Max Wind SpeedKm/h            1
Mean Win

In [4]:
# Drop the 'TravelExp_2021' column from the economic data
economic_data.drop('TravelExp_2021', axis=1, inplace=True)

# Verify the column is removed
print(economic_data.columns)


Index(['Country Code', 'GDP_2010', 'GDP_2011', 'GDP_2012', 'GDP_2013',
       'GDP_2014', 'GDP_2015', 'GDP_2016', 'GDP_2017', 'GDP_2018', 'GDP_2019',
       'GDP_2020', 'GDP_2021', 'Inflation_2010', 'Inflation_2011',
       'Inflation_2012', 'Inflation_2013', 'Inflation_2014', 'Inflation_2015',
       'Inflation_2016', 'Inflation_2017', 'Inflation_2018', 'Inflation_2019',
       'Inflation_2020', 'Inflation_2021', 'TravelExp_2010', 'TravelExp_2011',
       'TravelExp_2012', 'TravelExp_2013', 'TravelExp_2014', 'TravelExp_2015',
       'TravelExp_2016', 'TravelExp_2017', 'TravelExp_2018', 'TravelExp_2019',
       'TravelExp_2020'],
      dtype='object')


In [6]:
import pandas as pd


# Assuming your economic data columns are like 'GDP_2010', 'GDP_2011', ..., 'TravelExp_2020'
# We'll melt the dataframe to long format with one row per country per year
economic_data_long = economic_data.melt(id_vars=['Country Code'],
                                        var_name='Year_Variable',
                                        value_name='Value')

# Split 'Year_Variable' into separate 'Indicator' and 'Year'
economic_data_long['Indicator'] = economic_data_long['Year_Variable'].apply(lambda x: x.split('_')[0])
economic_data_long['Year'] = economic_data_long['Year_Variable'].apply(lambda x: x.split('_')[1])

# Pivot table to have indicators as columns again
economic_data_ready = economic_data_long.pivot_table(index=['Country Code', 'Year'],
                                                     columns='Indicator',
                                                     values='Value',
                                                     aggfunc='first').reset_index()

# Convert Year to integer for merging purposes
economic_data_ready['Year'] = economic_data_ready['Year'].astype(int)

print(economic_data_ready.head())


Indicator Country Code  Year       GDP  Inflation     TravelExp
0                  ABW  2010 -2.733456   2.078141  1.251000e+09
1                  ABW  2011  3.369238   4.316297  1.351000e+09
2                  ABW  2012 -1.040800   0.627472  1.402000e+09
3                  ABW  2013  6.431482  -2.372065  1.495000e+09
4                  ABW  2014 -1.586575   0.421441  1.605000e+09


In [7]:

# Convert 'Start date' to datetime and extract the year
travel_data['Start date'] = pd.to_datetime(travel_data['Start date'])
travel_data['Year'] = travel_data['Start date'].dt.year


In [9]:
# Print column names to verify
print("Travel Data Columns:", travel_data.columns)
print("Economic Data Columns:", economic_data_ready.columns)


Travel Data Columns: Index(['Month', 'Trip ID', 'Destination', 'Start date', 'End date',
       'Duration (days)', 'Traveler name', 'Traveler age', 'Traveler gender',
       'Traveler nationality', 'Accommodation type', 'Accommodation cost',
       'Transportation type', 'Transportation cost', 'Simplified Destination',
       'Max TemperatureC', 'Mean TemperatureC', 'Min TemperatureC',
       'Dew PointC', 'MeanDew PointC', 'Min DewpointC', 'Max Humidity',
       'Mean Humidity', 'Min Humidity', 'Max Sea Level PressurehPa',
       'Mean Sea Level PressurehPa', 'Min Sea Level PressurehPa',
       'Max VisibilityKm', 'Mean VisibilityKm', 'Min VisibilityKm',
       'Max Wind SpeedKm/h', 'Mean Wind SpeedKm/h', 'Max Gust SpeedKm/h',
       'Precipitationmm', 'CloudCover', 'WindDirDegrees', 'CustomerID',
       'ProdTaken', 'Age', 'TypeofContact', 'CityTier', 'DurationOfPitch',
       'Occupation', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups',
       'ProductPitched', 'PreferredPr

In [11]:
# Load datasets
travel_data = pd.read_csv('combined_travel_data.csv')

In [13]:
travel_data.describe(include= "all")

,Month,Trip ID,Destination,Start date,End date,Duration (days),Traveler name,Traveler age,Traveler gender,Traveler nationality,...,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
count,14353,14353.000000,14353,14353,14353,14353.000000,14353,14353.000000,14353,14353,...,14352,14352.000000,14352,14352.000000,14352.000000,14352.000000,14352.000000,14352.000000,14352,14352.000000
unique,44,NaN,59,111,125,NaN,109,NaN,2,41,...,5,NaN,4,NaN,NaN,NaN,NaN,NaN,5,NaN
top,2023-08-01,NaN,Bali,2023-08-15,2023-07-08,NaN,David Lee,NaN,Female,American,...,Basic,NaN,Married,NaN,NaN,NaN,NaN,NaN,Executive,NaN
freq,1144,NaN,833,624,520,NaN,832,NaN,7385,2704,...,5280,NaN,7008,NaN,NaN,NaN,NaN,NaN,5280,NaN
mean,NaN,69.992824,NaN,NaN,NaN,7.594231,NaN,33.116213,NaN,NaN,...,NaN,3.595318,NaN,3.206313,0.302536,3.036650,0.608696,1.200251,NaN,23597.082079
std,NaN,40.269833,NaN,NaN,NaN,1.590909,NaN,7.090660,NaN,NaN,...,NaN,0.808533,NaN,1.835893,0.459372,1.363048,0.488059,0.851633,NaN,5214.975226
min,NaN,1.000000,NaN,NaN,NaN,5.000000,NaN,20.000000,NaN,NaN,...,NaN,3.000000,NaN,1.000000,0.000000,1.000000,0.000000,0.000000,NaN,1000.000000
25%,NaN,35.000000,NaN,NaN,NaN,7.000000,NaN,28.000000,NaN,NaN,...,NaN,3.000000,NaN,2.000000,0.000000,2.000000,0.000000,1.000000,NaN,20580.750000
50%,NaN,70.000000,NaN,NaN,NaN,7.000000,NaN,31.000000,NaN,NaN,...,NaN,3.000000,NaN,3.000000,0.000000,3.000000,1.000000,1.000000,NaN,22347.000000
75%,NaN,105.000000,NaN,NaN,NaN,8.000000,NaN,38.000000,NaN,NaN,...,NaN,4.000000,NaN,4.000000,1.000000,4.000000,1.000000,2.000000,NaN,25453.000000


In [14]:
travel_data.isnull().sum()

Month                         0
Trip ID                       0
Destination                   0
Start date                    0
End date                      0
Duration (days)               0
Traveler name                 0
Traveler age                  0
Traveler gender               0
Traveler nationality          0
Accommodation type            0
Accommodation cost            0
Transportation type           0
Transportation cost           0
Simplified Destination        0
Max TemperatureC              1
Mean TemperatureC             1
Min TemperatureC              1
Dew PointC                    1
MeanDew PointC                1
Min DewpointC                 1
Max Humidity                  1
Mean Humidity                 1
Min Humidity                  1
Max Sea Level PressurehPa     1
Mean Sea Level PressurehPa    1
Min Sea Level PressurehPa     1
Max VisibilityKm              1
Mean VisibilityKm             1
Min VisibilityKm              1
Max Wind SpeedKm/h            1
Mean Win

In [15]:
travel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14353 entries, 0 to 14352
Data columns (total 56 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Month                       14353 non-null  object 
 1   Trip ID                     14353 non-null  int64  
 2   Destination                 14353 non-null  object 
 3   Start date                  14353 non-null  object 
 4   End date                    14353 non-null  object 
 5   Duration (days)             14353 non-null  float64
 6   Traveler name               14353 non-null  object 
 7   Traveler age                14353 non-null  float64
 8   Traveler gender             14353 non-null  object 
 9   Traveler nationality        14353 non-null  object 
 10  Accommodation type          14353 non-null  object 
 11  Accommodation cost          14353 non-null  float64
 12  Transportation type         14353 non-null  object 
 13  Transportation cost         143

In [16]:
travel_data.shape

(14353, 56)

In [17]:
travel_data.columns

Index(['Month', 'Trip ID', 'Destination', 'Start date', 'End date',
       'Duration (days)', 'Traveler name', 'Traveler age', 'Traveler gender',
       'Traveler nationality', 'Accommodation type', 'Accommodation cost',
       'Transportation type', 'Transportation cost', 'Simplified Destination',
       'Max TemperatureC', 'Mean TemperatureC', 'Min TemperatureC',
       'Dew PointC', 'MeanDew PointC', 'Min DewpointC', 'Max Humidity',
       'Mean Humidity', 'Min Humidity', 'Max Sea Level PressurehPa',
       'Mean Sea Level PressurehPa', 'Min Sea Level PressurehPa',
       'Max VisibilityKm', 'Mean VisibilityKm', 'Min VisibilityKm',
       'Max Wind SpeedKm/h', 'Mean Wind SpeedKm/h', 'Max Gust SpeedKm/h',
       'Precipitationmm', 'CloudCover', 'WindDirDegrees', 'CustomerID',
       'ProdTaken', 'Age', 'TypeofContact', 'CityTier', 'DurationOfPitch',
       'Occupation', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups',
       'ProductPitched', 'PreferredPropertyStar', 'Marital

In [18]:
travel_data.size

803768

In [19]:
travel_data.nunique()

Month                           44
Trip ID                        139
Destination                     59
Start date                     111
End date                       125
Duration (days)                  9
Traveler name                  109
Traveler age                    29
Traveler gender                  2
Traveler nationality            41
Accommodation type               8
Accommodation cost              25
Transportation type              9
Transportation cost             18
Simplified Destination          39
Max TemperatureC               110
Mean TemperatureC              110
Min TemperatureC               110
Dew PointC                     110
MeanDew PointC                 110
Min DewpointC                  110
Max Humidity                   110
Mean Humidity                  110
Min Humidity                   110
Max Sea Level PressurehPa      110
Mean Sea Level PressurehPa     110
Min Sea Level PressurehPa      110
Max VisibilityKm               110
Mean VisibilityKm   

In [21]:


# Extract unique destinations or countries from the travel data
unique_travel_countries = travel_data['Destination'].unique()  # or 'Country Code' if it exists
print("Unique Travel Destinations/Countries:")
print(unique_travel_countries)

# Extract unique country codes or names from the economic data
unique_economic_countries = economic_data['Country Code'].unique()  # or 'Country Name'
print("Unique Economic Country Codes:")
print(unique_economic_countries)


Unique Travel Destinations/Countries:
['London, UK' 'Phuket, Thailand' 'Bali, Indonesia' 'New York, USA'
 'Tokyo, Japan' 'Paris, France' 'Sydney, Australia'
 'Rio de Janeiro, Brazil' 'Amsterdam, Netherlands'
 'Dubai, United Arab Emirates' 'Cancun, Mexico' 'Barcelona, Spain'
 'Honolulu, Hawaii' 'Berlin, Germany' 'Marrakech, Morocco'
 'Edinburgh, Scotland' 'Paris' 'Bali' 'London' 'Tokyo' 'New York' 'Sydney'
 'Rome' 'Bangkok' 'Hawaii' 'Barcelona' 'Japan' 'Thailand' 'France'
 'Australia' 'Brazil' 'Greece' 'Egypt' 'Mexico' 'Italy' 'Spain' 'Canada'
 'New York City, USA' 'Bangkok, Thailand' 'Vancouver, Canada'
 'Sydney, AUS' 'Seoul, South Korea' 'Los Angeles, USA' 'Rome, Italy'
 'Cape Town' 'Cape Town, SA' 'Sydney, Aus' 'Bangkok, Thai' 'Phuket, Thai'
 'Dubai' 'Seoul' 'Rio de Janeiro' 'Amsterdam' 'Phuket' 'Santorini'
 'Phnom Penh' 'Athens, Greece' 'Cape Town, South Africa'
 'Auckland, New Zealand']
Unique Economic Country Codes:
['ABW' 'AFE' 'AFG' 'AFW' 'AGO' 'ALB' 'AND' 'ARB' 'ARE' 'ARG' 'ARM

In [22]:
# Convert lists to sets
set_travel_countries = set(unique_travel_countries)
set_economic_countries = set(unique_economic_countries)

# Find common and exclusive countries
common_countries = set_travel_countries.intersection(set_economic_countries)
exclusive_to_travel = set_travel_countries.difference(set_economic_countries)
exclusive_to_economic = set_economic_countries.difference(set_travel_countries)

print("Countries present in both datasets:")
print(common_countries)
print("Countries exclusive to travel data:")
print(exclusive_to_travel)
print("Countries exclusive to economic data:")
print(exclusive_to_economic)


Countries present in both datasets:
set()
Countries exclusive to travel data:
{'Hawaii', 'Thailand', 'Bangkok', 'Sydney, AUS', 'Australia', 'New York', 'Japan', 'Paris, France', 'Athens, Greece', 'Canada', 'Honolulu, Hawaii', 'London, UK', 'Rome', 'Greece', 'Seoul', 'Bangkok, Thai', 'Phnom Penh', 'Tokyo, Japan', 'Tokyo', 'Seoul, South Korea', 'France', 'Rio de Janeiro', 'Sydney, Australia', 'Dubai, United Arab Emirates', 'New York City, USA', 'Italy', 'Spain', 'Sydney', 'Barcelona', 'Marrakech, Morocco', 'Edinburgh, Scotland', 'Egypt', 'Phuket, Thai', 'Phuket', 'Cancun, Mexico', 'London', 'Auckland, New Zealand', 'Rio de Janeiro, Brazil', 'Amsterdam, Netherlands', 'New York, USA', 'Berlin, Germany', 'Bali', 'Mexico', 'Rome, Italy', 'Cape Town', 'Bangkok, Thailand', 'Barcelona, Spain', 'Amsterdam', 'Santorini', 'Bali, Indonesia', 'Vancouver, Canada', 'Los Angeles, USA', 'Cape Town, South Africa', 'Phuket, Thailand', 'Cape Town, SA', 'Brazil', 'Dubai', 'Paris', 'Sydney, Aus'}
Countries e

In [23]:
# Assuming your travel data is in a DataFrame called `travel_data`
travel_data['Country'] = travel_data['Destination'].apply(lambda x: x.split(', ')[-1])

# Now, get the unique country names from the travel data
unique_travel_countries = travel_data['Country'].unique()
print("Unique Travel Countries:", unique_travel_countries)


Unique Travel Countries: ['UK' 'Thailand' 'Indonesia' 'USA' 'Japan' 'France' 'Australia' 'Brazil'
 'Netherlands' 'United Arab Emirates' 'Mexico' 'Spain' 'Hawaii' 'Germany'
 'Morocco' 'Scotland' 'Paris' 'Bali' 'London' 'Tokyo' 'New York' 'Sydney'
 'Rome' 'Bangkok' 'Barcelona' 'Greece' 'Egypt' 'Italy' 'Canada' 'AUS'
 'South Korea' 'Cape Town' 'SA' 'Aus' 'Thai' 'Dubai' 'Seoul'
 'Rio de Janeiro' 'Amsterdam' 'Phuket' 'Santorini' 'Phnom Penh'
 'South Africa' 'New Zealand']


In [24]:
# Assuming your economic data country codes are stored in `unique_economic_countries`
set_travel_countries = set(unique_travel_countries)
set_economic_countries = set(unique_economic_countries)

# Find common and exclusive country entries
common_countries = set_travel_countries.intersection(set_economic_countries)
exclusive_to_travel = set_travel_countries.difference(set_economic_countries)
exclusive_to_economic = set_economic_countries.difference(set_travel_countries)

print("Common Countries:", common_countries)
print("Exclusive to Travel Data:", exclusive_to_travel)
print("Exclusive to Economic Data:", exclusive_to_economic)


Common Countries: {'AUS', 'USA'}
Exclusive to Travel Data: {'UK', 'Thailand', 'Bangkok', 'Australia', 'Hawaii', 'Morocco', 'SA', 'Thai', 'South Africa', 'New York', 'New Zealand', 'Japan', 'Canada', 'Aus', 'Rome', 'Germany', 'Greece', 'Seoul', 'Netherlands', 'Phnom Penh', 'Tokyo', 'France', 'Scotland', 'Indonesia', 'Rio de Janeiro', 'Italy', 'Spain', 'Sydney', 'Barcelona', 'United Arab Emirates', 'Phuket', 'Egypt', 'London', 'Mexico', 'Bali', 'Cape Town', 'Amsterdam', 'Santorini', 'Brazil', 'South Korea', 'Paris', 'Dubai'}
Exclusive to Economic Data: {'PRY', 'MHL', 'HUN', 'ISL', 'BHS', 'SOM', 'SDN', 'PRE', 'CHL', 'FCS', 'MWI', 'MOZ', 'KGZ', 'MNA', 'PSS', 'TLA', 'KNA', 'IND', 'VEN', 'ETH', 'NGA', 'CPV', 'TUR', 'MIC', 'HRV', 'LVA', 'OSS', 'GUM', 'ARM', 'KEN', 'CYP', 'SWE', 'UKR', 'SST', 'MLI', 'PRK', 'NLD', 'LTE', 'NRU', 'COM', 'LBN', 'TUV', 'FSM', 'DOM', 'IBD', 'CUB', 'FRO', 'GRC', 'MAC', 'IDB', 'UZB', 'CUW', 'KIR', 'LMC', 'MUS', 'CIV', 'BRB', 'MDV', 'LKA', 'BMU', 'MEX', 'MCO', 'SYR', '

In [25]:
# Expanded dictionary mapping from country name variations in travel data to ISO country codes
country_code_mapping = {
    'UK': 'GBR',
    'Thailand': 'THA',
    'Indonesia': 'IDN',
    'USA': 'USA',
    'Japan': 'JPN',
    'France': 'FRA',
    'Australia': 'AUS',
    'Brazil': 'BRA',
    'Netherlands': 'NLD',
    'United Arab Emirates': 'ARE',
    'Mexico': 'MEX',
    'Spain': 'ESP',
    'Hawaii': 'USA',  # Assuming Hawaii as part of the USA in travel context
    'Germany': 'DEU',
    'Morocco': 'MAR',
    'Scotland': 'GBR',
    'Italy': 'ITA',
    'Greece': 'GRC',
    'Egypt': 'EGY',
    'Canada': 'CAN',
    'South Korea': 'KOR',
    'New Zealand': 'NZL',
    'Paris': 'FRA',
    'London': 'GBR',
    'Tokyo': 'JPN',
    'New York': 'USA',
    'Sydney': 'AUS',
    'Rome': 'ITA',
    'Bangkok': 'THA',
    'Barcelona': 'ESP',
    'Dubai': 'ARE',
    'Rio de Janeiro': 'BRA',
    'Amsterdam': 'NLD',
    'Phuket': 'THA',
    'Santorini': 'GRC',
    'Phnom Penh': 'KHM',
    'Cape Town': 'ZAF',
    'Bali': 'IDN',
    'Seoul': 'KOR'
    # Add more mappings as needed
}

# Applying the mapping to the 'Destination' column in the travel data
travel_data['Country Code'] = travel_data['Destination'].apply(
    lambda dest: country_code_mapping.get(dest.split(', ')[-1], None)
)

# Ensure no null values in 'Country Code' if necessary, handle or drop them
print(travel_data['Country Code'].isnull().sum())

# Merge the travel data with economic data on 'Country Code'
combined_data = pd.merge(
    travel_data, 
    economic_data, 
    on='Country Code', 
    how='left'
)

# Check the head of the combined dataframe to ensure correctness
print(combined_data.head())


832
        Month  Trip ID Destination  Start date    End date  Duration (days)  \
0  2023-05-01        1  London, UK  2023-05-01  2023-05-08              7.0   
1  2023-05-01        1  London, UK  2023-05-01  2023-05-08              7.0   
2  2023-05-01        1  London, UK  2023-05-01  2023-05-08              7.0   
3  2023-05-01        1  London, UK  2023-05-01  2023-05-08              7.0   
4  2023-05-01        1  London, UK  2023-05-01  2023-05-08              7.0   

  Traveler name  Traveler age Traveler gender Traveler nationality  ...  \
0    John Smith          35.0            Male             American  ...   
1    John Smith          35.0            Male             American  ...   
2    John Smith          35.0            Male             American  ...   
3    John Smith          35.0            Male             American  ...   
4    John Smith          35.0            Male             American  ...   

  TravelExp_2011  TravelExp_2012 TravelExp_2013  TravelExp_2014  \
0  